In [5]:
import os


from getpass import getpass
from coreapi.auth import TokenAuthentication
from qiskit import BasicAer, execute
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit
from query import query
from quantuminspire.credentials import get_authentication
from quantuminspire.qiskit import QI


QI_URL = os.getenv('API_URL', 'https://api.quantum-inspire.com/')
authentication = TokenAuthentication('40a1a77810b8d0e10428efa64ae124e79f2b6336', scheme='token')
QI.set_authentication(authentication, QI_URL)
qi_backend = QI.get_backend('QX single-node simulator')


q = QuantumRegister(12, "q")
c = ClassicalRegister(8, "c")
qc = QuantumCircuit(q, c, name="conditional")

qc.x([0,2,3])
qc=qc.compose(query().to_gate(label="query"),range(12))

qc.measure(q[4:12],c[0:8])

qi_job = execute(qc, backend=qi_backend, shots=128)
qi_result = qi_job.result()
histogram = qi_result.get_counts(qc)
print("\nResult from the remote Quantum Inspire backend:\n")
print('State\tCounts')
[print('{0}\t{1}'.format(state, counts)) for state, counts in histogram.items()]

print("\nResult from the local Qiskit simulator backend:\n")
backend = BasicAer.get_backend("qasm_simulator")
job = execute(qc, backend=backend, shots=128)
result = job.result()
print(result.get_counts(qc))
print(qc.draw(output="text"))


Result from the remote Quantum Inspire backend:

State	Counts
00100011	128

Result from the local Qiskit simulator backend:

{'00100011': 128}
      ┌───┐┌─────────┐                        
 q_0: ┤ X ├┤0        ├────────────────────────
      └───┘│         │                        
 q_1: ─────┤1        ├────────────────────────
      ┌───┐│         │                        
 q_2: ┤ X ├┤2        ├────────────────────────
      ├───┤│         │                        
 q_3: ┤ X ├┤3        ├────────────────────────
      └───┘│         │┌─┐                     
 q_4: ─────┤4        ├┤M├─────────────────────
           │         │└╥┘┌─┐                  
 q_5: ─────┤5        ├─╫─┤M├──────────────────
           │   query │ ║ └╥┘┌─┐               
 q_6: ─────┤6        ├─╫──╫─┤M├───────────────
           │         │ ║  ║ └╥┘┌─┐            
 q_7: ─────┤7        ├─╫──╫──╫─┤M├────────────
           │         │ ║  ║  ║ └╥┘┌─┐         
 q_8: ─────┤8        ├─╫──╫──╫──╫─┤M├─────────
          

SyntaxError: invalid syntax (Temp/ipykernel_204/2151542490.py, line 1)